In [ ]:
from google.colab import drive

drive.mount('/content/drive')
root_dir = "/content/drive/My Drive/"
base_dir = root_dir + 'Tesi/'

In [ ]:
!pip install rake-nltk

In [ ]:
import nltk 
nltk.download('stopwords')

In [ ]:
from itertools import chain
from collections import Counter

In [ ]:
import pandas as pd 

df = pd.read_csv(base_dir + 'downstream/ccoha2_preprocessed.csv')

In [ ]:
df.shape

RAKE

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from rake_nltk import Rake

r = Rake(min_length=2, max_length=4, language='english') # Uses stopwords for english from NLTK, and all puntuation characters.

In [ ]:
def rake_implement(x,r):
     r.extract_keywords_from_text(x)
     return r.get_ranked_phrases()

df['keywords'] = df['text'].apply(lambda x: rake_implement(x,r))

MWE THRESHOLD

In [ ]:
mwe_counter = Counter(chain.from_iterable(df['keywords'])) 

In [ ]:
df['keywords'] = [([j for j in i if mwe_counter[j] >= 5]) for i in df['keywords']]

SUBSTITUTION

In [ ]:
import re

for i in range(len(df)):
  df['text'][i] = re.sub('('+'|'.join('\\b'+re.escape(g)+'\\b' for g in df['keywords'][i])+')', lambda m: m.group(0).replace(' ', '_'), df['text'][i], flags=re.I)

In [ ]:
df

In [ ]:
compression_opts = dict(method='zip', archive_name='RakeKeywords.csv')

df['keywords'].to_csv('RakeKeywords.zip', index=False, compression=compression_opts)

In [ ]:
compression_opts = dict(method='zip', archive_name='newCCOHA2_Rake.csv')

df['text'].to_csv('newCCOHA2_Rake.zip', index=False, compression=compression_opts)

In [ ]:
compression_opts = dict(method='zip', archive_name='preprocessed_Rake.csv')

df.to_csv('preprocessed_Rake.zip', index=False, compression=compression_opts)

TRAINING

In [ ]:
for i in range(len(df)):
  df['text'][i]=df['text'][i].lower()

In [ ]:
df['token']=""

In [ ]:
def remove_p(text):
    text  = "".join([char for char in text if char not in punct])
    return text

df['token'] = df['text'].apply(lambda x: remove_p(x))

In [ ]:
# tokenize sentences in corpus
wpt = nltk.WordPunctTokenizer()
tokenized_corpus = [wpt.tokenize(document) for document in df['text']]

In [ ]:
from gensim.models import Word2Vec

In [ ]:
w2v_model = Word2Vec(size=300,
                     negative=5,
                     workers=5)

In [ ]:
w2v_model.build_vocab(tokenized_corpus)

In [ ]:
w2v_model.train(tokenized_corpus, total_examples=w2v_model.corpus_count, epochs=50, report_delay=1)

In [ ]:
w2v_model.wv.most_similar(positive=["new_york"])

In [ ]:
w2v_model.wv.doesnt_match(['car', 'new_york', 'city'])

In [ ]:
w2v_model.wv.save_word2vec_format('newText_Rake.txt', binary=False)